In [1]:
import cv2
import os
from math import ceil
import time
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import random
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib as mpl

# mpl.rcParams['animation.ffmpeg_path_vid_vid_vid_vid'] = r'C:\\ffmpeg\\bin\\ffmpeg.exe'
import matplotlib.gridspec as gridspec

from koniq_api.lib import MyModel

import os
import sys
import pandas as pd

2022-04-12 21:57:05.642831: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alexkkir/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-12 21:57:05.642900: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loaded Kuti


In [2]:
subjective_csv = 'subjective_scores.csv'

def parse(name: str) -> dict[str]:
    out = dict()
    name = name.split('.')[0]
    name = name.split('enc_res_')[1]

    str1 = '_mv_offline_2k_v1_'
    str2 = '_mv_offline_2k_'

    if name.find(str1) >= 0:
        method = 'mv_offline_2k_v1'
        name = name.split(str1)
    else:
        method = 'mv_offline_2k'
        name = name.split(str2)

    out['method'] = method
    out['codec'] = name[0]

    name = name[1]
    name = name.split('_')
    out['crf'] = int(name[-1])

    name = '_'.join(name[0:-1])
    out['name'] = name
    return out

def get_1_score(df: pd.DataFrame, name, codec, crf, method):
    x = df[(df['comparison'] == 'ugc') & (df['sequence'] == name) & (df['crf'] == int(crf)) & (df['codec'] == codec) & (
            df['preset'] == method)]
    x = x['subjective']

    if len(x) == 1:
        x = float(x)
    else:
        return None
    
    return x

def get_subj_scores(videos):
    subj = pd.read_csv(subjective_csv, sep=';')
    return [get_1_score(subj, **parse(name)) for name in videos]

In [3]:
file = 'hair_7.npy'
folder = 'blue_hair_x265/'
videos = sorted(os.listdir(folder))

bscores = []
subjs = get_subj_scores(videos)

with open(file, 'rb') as f:
    try:
        while True:
            bscores.append(np.load(f))
    except:
        pass


videos = videos[0:len(bscores)]
subjs = subjs[0:len(bscores)]
assert len(videos) == len(bscores) == len(subjs)

In [4]:
print(sys.getsizeof(bscores * bscores[0]))

268952


In [5]:
print(videos)
print(len(bscores))
print(subjs)

['enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_33.mp4', 'enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_38.mp4', 'enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_43.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_33.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_39.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_45.mp4', 'enc_res_kingsoft_v3_mv_offline_2k_blue_hair_28.mp4']
7
[5.673068297172945, 5.66340286203501, 5.243526334400184, 6.346977572482853, 6.177970683278712, 5.417861161606206, None]


In [6]:
deleted = 0
for i in range(len(subjs)):
    if subjs[i - deleted] is None:
        subjs.pop(i - deleted)
        videos.pop(i - deleted)
        bscores.pop(i - deleted)
        deleted += 1

print(videos)
print(len(bscores))
print(subjs)

['enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_33.mp4', 'enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_38.mp4', 'enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_43.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_33.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_39.mp4', 'enc_res_kingsoft_v2_mv_offline_2k_blue_hair_45.mp4']
6
[5.673068297172945, 5.66340286203501, 5.243526334400184, 6.346977572482853, 6.177970683278712, 5.417861161606206]


In [7]:
MAX = 2

class MetricArray:
    def __init__(self, vid_path, mask_path, n_height, n_width):
        self.vid = None if vid_path is None else VideoReader(vid_path)
        self.n_height = n_height
        self.n_width = n_width
        self.block_height = self.vid.height // n_height
        self.block_width = self.vid.width // n_width
        self.scores = np.zeros((self.vid.length, self.n_height, self.n_width))
        self.mask_raw = None if mask_path is None else VideoReader(mask_path)
        self.metric = MODEL
        self.mask = np.zeros_like(self.scores)
        self.length = self.vid.length

    def get_subframe(self, frame, i, j) -> np.array:
        return frame[
               i * self.block_height: min(self.vid.height, (i + 1) * self.block_height),
               j * self.block_width: min(self.vid.width, (j + 1) * self.block_width)
               ]

    def split_video(self):
        """
        devide video on n * m subvideos and save each of them in ./tmp/ folder
        :return:
        """

        TMP_DIR = './tmp/'
        if not os.path.exists(TMP_DIR): 
            os.mkdir(TMP_DIR)
        for f in os.listdir(TMP_DIR):
            os.remove(TMP_DIR + f)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        for i in range(self.n_height):
            for j in range(self.n_width):
                out = cv2.VideoWriter(f'{TMP_DIR}{i}_{j}.mp4', fourcc, vid.fps, (self.block_width, self.block_height))
                for frame in vid:
                    subframe = self.get_subframe(frame, i, j)
                    out.write(subframe)
                out.release()
                vid.refresh()
                
    
    def calc_raw_metric(self):
        for frame_idx, frame in tqdm(enumerate(self.vid), total=self.length):
            for i in range(self.n_height):
                for j in range(self.n_width):
                    subframe = self.get_subframe(frame, i, j)
                    score = self.metric(subframe)
                    self.scores[frame_idx][i][j] = score
    
    def resize_mask(self):
        self.mask_raw.refresh()
        for fi, frame in enumerate(self.mask_raw):
            for i in range(self.mask.shape[1]):
                for j in range(self.mask.shape[2]):
                    self.mask[fi][i][j] = frame[ 
                        i * self.block_height : (i + 1) * (self.block_height ), 
                        j * self.block_width: (j + 1) * (self.block_width)].mean() / 255     
        self.mask_raw.refresh()


    def apply_mask(self):
        return self.scores * self.mask
    
    def calc_metric(self):
        return self.apply_mask().mean()

In [20]:
def calc_metric_dummy_all_together(m: MetricArray) -> float:
    new_mask = m.mask.copy()
    res = 0
    for mask, metric in zip(new_mask, m.scores):
        res += (mask * metric).mean()
    res /= m.length
    return res

def calc_metric_dummy_by_frame(m: MetricArray) -> float:
    new_mask = m.mask.copy()
    res = 0
    for mask, metric in zip(new_mask, m.scores):
        res += (mask * metric).sum() / mask.sum()
    res /= m.length
    return res

def calc_metric_norm(m: MetricArray) -> float:
    new_mask = m.mask.copy()
    for i in range(m.length):
        frame = new_mask[i]
        frame -= frame.mean()
        frame /= frame.std()
        frame += 1
        new_mask[i] = frame
        # print(frame.mean(), frame.mean(), frame.std())
    res = 0
    for mask, metric in zip(new_mask, m.scores):
        res += (mask * metric).sum() / mask.sum()
    res /= m.length
    return res

def calc_metric_sum(m):
    return m.scores.sum()

In [9]:
class VideoReader:
    def __init__(self, path):
        self.path = path
        self.cap = cv2.VideoCapture(path)
        self.height = int(self.cap.get(4))
        self.width = int(self.cap.get(3))
        self.it = iter(self)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

    def __iter__(self):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret == False:
                break
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            yield frame

    def __call__(self):
        return next(self.it)

    def __del__(self):
        self.cap.release()

    def refresh(self):
        self.cap = cv2.VideoCapture(self.path)

In [10]:
MODEL = MyModel()

Loading model InceptionResNetV2


2022-04-12 21:57:38.339775: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-12 21:57:38.339817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexkkir-Extensa-215-51G): /proc/driver/nvidia/version does not exist
2022-04-12 21:57:38.340585: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
def calc_pcc(x, y):
    return np.corrcoef(x, y)

In [17]:
scores = []
m = MetricArray(folder + videos[0], 'hair_mask.mp4', 4, 4)
m.resize_mask()
for video, bscore in zip(videos, bscores):
    m.scores = bscore
    score = calc_metric_sum(m)
    scores.append(score)
    
print(scores)
print(subjs)
print(calc_pcc(scores, subjs))

[63223.899787813425, 65720.45207762718, 62683.14124214649, 56498.16049385071, 63344.35722601414, 62464.953249424696]
[5.673068297172945, 5.66340286203501, 5.243526334400184, 6.346977572482853, 6.177970683278712, 5.417861161606206]
[[ 1.         -0.54983594]
 [-0.54983594  1.        ]]


In [21]:
scores = []
m = MetricArray(folder + videos[0], 'hair_mask.mp4', 4, 4)
m.resize_mask()
for video, bscore in zip(videos, bscores):
    m.scores = bscore
    score = calc_metric_dummy_all_together(m)
    scores.append(score)
    
print(scores)
print(subjs)
print(calc_pcc(scores, subjs))

[0.38539724256919017, 0.4554052083257252, 0.40356824182723344, 0.33718243308630774, 0.45256664521129236, 0.43741904134237386]
[5.673068297172945, 5.66340286203501, 5.243526334400184, 6.346977572482853, 6.177970683278712, 5.417861161606206]
[[ 1.         -0.33686407]
 [-0.33686407  1.        ]]
